In [1]:
import sys
sys.path.append('../')

In [2]:
from application.name_obj_classes import PubName, PersonName, remove_punct

In [3]:
from application.review_obj_class import ReviewObj

In [4]:
from application.text_preprocessing import preprocess_text

In [5]:
import os
import pandas as pd
import re
from collections import Counter
import numpy as np
import pickle
from nltk.metrics import edit_distance
%pprint

Pretty printing has been turned OFF


In [6]:
# loading in files
directory = "../../aps_reviews_50/aps_reviews/"
filenames = os.listdir(directory)
txts = []
for file in filenames:
    with open(directory + file) as f:
        txts.append(f.read())

In [7]:
[x.split('.')[0] for x in filenames]

['136726613', '90142613', '90390665', '124486140', '126315326', '136992742', '137651039', '89668273', '136971619', '90313218', '128376070', '124688361', '124882533', '89639059', '88461015', '124190789', '136902134', '90180084', '137244878', '88721263', '137412078', '89675095', '760950213', '124217248', '853770606', '137195890', '125727674', '88797650', '89649761', '90967694', '89672866', '89674750', '89635917', '124196531', '128176513', '88477095', '125715359', '124189638', '136887509', '90554411', '90186165', '125502663', '125724646', '124740955', '124707428', '89644579', '137072896', '136685639', '126788760', '89671107']

## Part 1: Publisher Names

In [8]:
def clean_pub_matches(match_list):
    cleaned_matches = []
    for match in match_list:
        index_to_start = 0
        for i, x in enumerate(match[1].split()):
            if x[0].islower() and x[0]!='&':
                index_to_start = i+1
        cleaned_matches.append(' '.join(match[1].split()[index_to_start:]))
    return cleaned_matches

In [9]:
def remove_dash_for_pub(pub_match):
    return re.sub(r'(?<!/w)-(?!/w)', '', pub_match)

In [10]:
pub_ends = ['company','co','incorporated','inc','firm','press','group','publishers','publishing',
                    'publications','pub','books','ltd','limited','society','house','associates']

In [11]:
pub_ends_list = '|'.join([x.capitalize()+'\.?(?!\w)' for x in pub_ends])

In [12]:
pub_ends = [x.capitalize() for x in pub_ends]

In [13]:
pub_ends

['Company', 'Co', 'Incorporated', 'Inc', 'Firm', 'Press', 'Group', 'Publishers', 'Publishing', 'Publications', 'Pub', 'Books', 'Ltd', 'Limited', 'Society', 'House', 'Associates']

``` 
    for e, index in enumerate(indices):
        
        if (e==len(indices)-1):
            end_index = -1
        else:
            end_index = indices[e+1][0]
        
        end_span = len(txt[indices[e][0]:end_index])   
        get_match = re.finditer('[A-Z]\w+[^A-Z]|[A-Z].[^A-Z]', txt[indices[e][0]:end_index])
        matches = [(m.span(), m.group()) for m in get_match]
        matches.reverse()
        
        for n, m in enumerate(matches):
            if n<len(matches)-1:
                if (m[0][1] != matches[n-1][0][0]):
                    end_span = m[0][1]
        
        result = txt[indices[e][0]:(indices[e][0] + end_span - 1)]
        
        if len(result) > len(indices[e][1]):
            names.append(txt[indices[e][0]:(indices[e][0] + end_span - 1)])
            spans.append(indices[e][0])
```

+ split into tokens
+ stop as soon 
+ stop as city names
+ put word tokenizer in review obj

In [14]:
known_publishers = ["Charles Scribner's Sons","Scribner","Macmillan","Funk & Wagnalls","McClure, Philips","Houghton Mifflin","G.P. Putnam's Sons", "G.P. Putnam",
 "Harper & Brothers","Harper","J.B. Lippincott","J. B. Lippincott", "Doubleday, Page","Doubleday","D. Appleton","Longmans, Green",
 "Longman","Henry Holt","Holt","Adam & Charles Black"]

In [15]:
review_list = []
for filename, txt in zip([x.split('.')[0] for x in filenames], txts):
    review_list.append(ReviewObj(filename, txt))

In [16]:
toks = review_list[24].cleaned_toks

In [39]:
temp_text = review_list[24].cleaned_text

In [55]:
def obscure_single_match(text, x, y):
    text_list = list(text)
    text_list[x:y] = list(len(text[x:y]) * '@')
    return ''.join(text_list)

In [57]:
obscure_single_match(temp_text, *(70,82))

"LITERARY System of Measurments adopted by the National Association of @@@@@@@@@@@@ Painters and Decorators of the United States 9x12 in 60 pp price 00 The Painters ' Magazine This is timely book and one that should be in the hands of every builder and contractor as one of the aids to correct estimating for painting It is noted fact that heretofore two painters measuring off any given piece of work have been sure to arrive at different results owing to the fact that no uniform rules of measurements have been in vogue The work before us is an en deavor and successful one we think towards establishing an uniform method of measuring off work and as it has been prepared under the di rection of The National Association of Master House Painters and Decorators of the United States it carries with it the impress of authority and stands as an official guide for master and man in making measurments of work done or work contemplated The book is oblong 9x12 inches and is illustrated with six litho

In [56]:
for x in review_list[24].pub_names:
    print(x.review_loc)
    tbr = temp_text[x.review_loc[0]:x.review_loc[1]]
    tr = len(tbr) * '@'
    print(tbr)
    print(tr)
    temp_text.replace(tbr, tr, 1)

(70, 82)
Master House
@@@@@@@@@@@@
(70, 82)
Master House
@@@@@@@@@@@@


In [51]:
temp_text

"LITERARY System of Measurments adopted by the National Association of Master House Painters and Decorators of the United States 9x12 in 60 pp price 00 The Painters ' Magazine This is timely book and one that should be in the hands of every builder and contractor as one of the aids to correct estimating for painting It is noted fact that heretofore two painters measuring off any given piece of work have been sure to arrive at different results owing to the fact that no uniform rules of measurements have been in vogue The work before us is an en deavor and successful one we think towards establishing an uniform method of measuring off work and as it has been prepared under the di rection of The National Association of Master House Painters and Decorators of the United States it carries with it the impress of authority and stands as an official guide for master and man in making measurments of work done or work contemplated The book is oblong 9x12 inches and is illustrated with six litho

```
def __obscure_matches(self, name = 'ex'):
    text_list = list(self.cleaned_text)
    if name == 'pub':
        for (x, y) in [pub.review_loc for pub in self.pub_names]:
                text_list[x:y] = list(len(self.cleaned_text[x:y]) * '@')
        if name == 'person':
            for (x, y) in [pers.review_loc for pers in self.person_names]:
                text_list[x:y] = list(len(self.cleaned_text[x:y]) * '@')
        return ''.join(text_list)
```

In [17]:
toks

['LITERARY', 'System', 'of', 'Measurments', 'adopted', 'by', 'the', 'National', 'Association', 'of', 'Master', 'House', 'Painters', 'and', 'Decorators', 'of', 'the', 'United', 'States', '9x12', 'in', '60', 'pp', 'price', '00', 'The', 'Painters', "'", 'Magazine', 'This', 'is', 'timely', 'book', 'and', 'one', 'that', 'should', 'be', 'in', 'the', 'hands', 'of', 'every', 'builder', 'and', 'contractor', 'as', 'one', 'of', 'the', 'aids', 'to', 'correct', 'estimating', 'for', 'painting', 'It', 'is', 'noted', 'fact', 'that', 'heretofore', 'two', 'painters', 'measuring', 'off', 'any', 'given', 'piece', 'of', 'work', 'have', 'been', 'sure', 'to', 'arrive', 'at', 'different', 'results', 'owing', 'to', 'the', 'fact', 'that', 'no', 'uniform', 'rules', 'of', 'measurements', 'have', 'been', 'in', 'vogue', 'The', 'work', 'before', 'us', 'is', 'an', 'en', 'deavor', 'and', 'successful', 'one', 'we', 'think', 'towards', 'establishing', 'an', 'uniform', 'method', 'of', 'measuring', 'off', 'work', 'and', '

In [18]:
city_dict = pickle.load(open('../data/city_dict.pkl', 'rb'))

In [19]:
from symspellpy.symspellpy import SymSpell, Verbosity

In [20]:
def is_part_of_pub(pub_part):
    if (pub_part == 'and') or (pub_part =='&'):
        return True
    elif city_dict.lookup(pub_part.lower(), Verbosity.CLOSEST, max_edit_distance=1):
        return False
    else:
        return pub_part[0].isupper()

In [21]:
is_part_of_pub('Egg')

True

In [22]:
pubnames = []
for e, tok in enumerate(toks):
    if tok.replace(",","").replace(".","").replace('-',"") in pub_ends:
        if is_part_of_pub(toks[e-1]):
            pub_name = [tok]
            pub_span = []
            for pos in range(e-1, e-6, -1):
                if toks[pos] == '.':
                    break
                elif not is_part_of_pub(toks[pos]):
                    break
                pub_name.append(toks[pos])
                pub_span.append(pos)   
            pubnames.append((pub_span[-1], e+1))

In [23]:
pubnames

[(10, 12), (130, 132)]

In [24]:
def remove_duplicate_pubnames(pnlist):
    cleaned = []
    for e, (x, y) in enumerate(pnlist):
        starts = [a for (a,b) in pnlist[e+1:]]
        if x in starts:
            pass
        else:
            cleaned.append(pnlist[e])
    return cleaned

In [25]:
cleaned = []
for e, (x, y) in enumerate(pubnames):
    starts = [a for (a,b) in pubnames[e+1:]]
    if x in starts:
        pass
    else:
        cleaned.append(pubnames[e])

In [26]:
remove_duplicate_pubnames(pubnames)

[(10, 12), (130, 132)]

In [27]:
for (x, y) in pubnames:
    newname = ' '.join(toks[x:y])
    print(' '.join(toks[x:y]))
    match = re.search(newname, review_list[0].cleaned_text)
    print(match.span())

Wm Wood Co


AttributeError: 'NoneType' object has no attribute 'span'

In [ ]:
any([x.isalpha() for x in ['Cassel','&','Co.']])

In [28]:
def get_publishers(review):
    """
    Takes a ReviewObj. 
    Returns a list of potential publishers. Searches using pub_ends, capitalization, and associates.
    
    For reference:
    -------------
    pub_ends = ['co','company','inc','incorporated','firm','press','group', 'pub','publishers','publishing',
                    'publications','books','ltd','limited','society','house','associates']
                    
    pub_associates = ['sons','son','brother','brothers']
    
    """
    
    pubs = []
    spans = []
    
    toks = review.cleaned_toks
    txt = review.cleaned_text
    
    pubnames = []
    
    for e, tok in enumerate(toks):
        if tok.replace(",","").replace(".","") in pub_ends:
            if is_part_of_pub(toks[e-1]):
                pub_name = []
                pub_span = []
                for pos in range(e-1, e-6, -1):
                    if toks[pos] == '.':
                        break
                    if not is_part_of_pub(toks[pos]):
                        break
                    pub_name.append(toks[pos])
                    pub_span.append(pos)
                if any([x.isalpha() for x in [word for word in pub_name if word !='and']]) and any([len(x)>2 for x in [word for word in pub_name if word !='and']]):
                    pubnames.append((pub_span[-1], e+1))
    
    if len(pubnames) > 0:
        for (x, y) in pubnames:
            newname = ' '.join(toks[x:y])
            pubs.append(newname)
            match = re.search(newname, txt)
            spans.append(match.span())

    pubs = [PubName(word) for word in pubs]
    
    for pub in pubs:
        pub.review_id = review.review_id
    
    for e, pub in enumerate(pubs):
        pub.review_id = review.review_id
        pub.review_loc = spans[e]
    
    return pubs

In [29]:
for filename, txt in zip([x.split('.')[0] for x in filenames],txts):
    rev = ReviewObj(filename, txt)
    print(filename)
    print(get_publishers(rev))
    print()

136726613
['Cassell & Co.', 'E. P. Dutton & Co.', 'Thomas Y. Crowell & Co.', 'American Historical Society', 'Dunlap Society']

90142613
[]

90390665
[]

124486140
[]

126315326
[]

136992742
['Filtlh Co']

137651039
['KnRR & Co', 'PUBLrISHIING Co', 'American Geographical Society', 'B. LIPizxcoTr & Co', 'B. LpnIrrcorr & Co.']

89668273
[]

136971619
[]

90313218
[]

128376070
[]

124688361
[]

124882533
[]

89639059
[]

88461015
['Blakiston Co', 'Robert Clarke Co', 'Graphic Company']

124190789
['Century Co.', 'Slead & Co.', 'B. Lippincott Co.', 'Macmillan Co.', 'Maemillall Co.']

136902134
['Sou & Co.']

90180084
['Macmillan & Co.', 'Mifflin & Co.']

137244878
[]

88721263
[]

137412078
[]

89675095
[]

760950213
[]

124217248
['B. Lippincott Co.', 'Macmillan & Co.', 'Friend Society']

853770606
['Master House', 'Master House']

137195890
[]

125727674
[]

88797650
['Funk and Wagnails Company', 'Outing Publishing', 'Outing Publishing Company', 'Open Court Pubpishing Co', 'Sheet Metal P

In [30]:
for filename, txt in zip([x.split('.')[0] for x in filenames],txts):
    rev = ReviewObj(filename, txt)
    print(filename)
    if rev.person_names:
        for x in rev.pub_names:
            print(x)
        print()

136726613
Cassell & Co.
E. P. Dutton & Co.
Thomas Y. Crowell & Co.
American Historical Society
Dunlap Society

90142613

90390665

124486140

126315326
136992742
137651039
KnRR & Co
PUBLrISHIING Co
American Geographical Society
B. LIPizxcoTr & Co
B. LpnIrrcorr & Co.

89668273

136971619
90313218

128376070

124688361
124882533

89639059

88461015
Blakiston Co
Robert Clarke Co
Graphic Company

124190789
Century Co.
Slead & Co.
B. Lippincott Co.
Macmillan Co.
Maemillall Co.

136902134
Sou & Co.

90180084
Macmillan & Co.
Mifflin & Co.

137244878

88721263
137412078

89675095

760950213
124217248
B. Lippincott Co.
Macmillan & Co.
Friend Society

853770606
Master House
Master House

137195890

125727674

88797650
Funk and Wagnails Company
Outing Publishing Company
Open Court Pubpishing Co
Sheet Metal Publishing Company

89649761
Macmillan & Co.

90967694

89672866

89674750
Shakespearean Society.
E. P. Dutton & Co.
Maynard & Co.

89635917
Macmillan & Co.

124196531

128176513
William Wood C

## Part 1: Person Names

In [31]:
titles = """Doctor,Dr,Mr,Mrs,Miss,Msgr,Monsignor,Rev,Reverend,Hon,Honorable,Honourable,Prof,Professor,Madame,Madam,Lady,Lord,Sir,Dame,Master,Mistress,Princess,Prince,Duke,Duchess,Baron,Father,Chancellor,Principal,President,Pres,Warden,Dean,Regent,Rector,Provost,Director
"""

In [32]:
titles = titles.rstrip().split(',')

In [33]:
title_list = '\.?\s(?=[A-Z])|'.join(titles)

In [34]:
def remove_punct_not_following_title_or_initial(name):
    name_parts = name.split()
    cleaned_name = []
    for part in name_parts:
        if part[-1] in '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~':
            if (len(part)>2) and (part[:-1] not in titles):
                cleaned_name.append(part[:-1])
            else:
                cleaned_name.append(part)
        else:
            cleaned_name.append(part)
    return ' '.join(cleaned_name)

In [35]:
def clean_name(name):
    name = remove_punct_not_following_title_or_initial(name)
    cleaned_name = []
    return ' '.join([word for word in name.split() if (word[0].isalpha())])

In [36]:
def getCapitalizedWords(txt):
    """
    Returns strings of capitalized words up 3 words long. 
    Removes words/phrases containing stopwords and words found later in the text lowercased.
    """
    #listen idk why it won't just let me put in an optional repeat either
    all_words = []
    
    #three words
    all_words.extend([match for match in re.findall('[A-Z]\S* [A-Z]\S* [A-Z]\S+', txt) if 
                      all([(remove_punct(word).lower() not in stopword_list) for word in match.split()]) 
                      and all([(remove_punct(word).lower() not in txt) for word in match.split()])])
    
    #two words
    two_words = [match for match in re.findall('[A-Z]\S* [A-Z]\S+', txt) if 
                      all([(remove_punct(word).lower() not in stopword_list) for word in match.split()]) 
                      and all([(remove_punct(word).lower() not in txt) for word in match.split()])
                      and all([match not in x for x in all_words])]
    
    all_words.extend(two_words)
    
    #one word
    one_words = [match for match in re.findall('[A-Z]\S+', txt) if 
                      (remove_punct(match).lower() not in stopword_list) 
                      and (remove_punct(match).lower() not in txt)
                      and all([match not in x for x in all_words])]
    
    all_words.extend(one_words)
    
    return [word for word in [' '.join([removePunct(y) for y in x.split() if remove_punct(y) not in titles]) 
                      for x in all_words] if (len(word)>1)]

In [37]:
def consolidateNames(name_list):
    """
    Takes list of AuthNames and returns list of lists, consolidating by likely identical authors.
    """
    name_set = []
    used_indices = []
    last_names = sorted([name.last_name for name in name_list], key=len)
    
    for i, name in enumerate(last_names):
        if i not in used_indices:
            full_name = [x for x in name_list if x.last_name == name][0]
            name_holder = [full_name]
            for j, name2 in enumerate(last_names):
                full_name2 = [x for x in name_list if x.last_name == name2][0]
                if (i < (len(last_names) - 1)) and (i!=j):
                    if (edit_distance(name, name2[:len(name)+1]) < 2) and (j not in used_indices):
                        if (full_name.first_initial==full_name2.first_initial and full_name.middle_initial==full_name2.middle_initial or full_name.title==full_name2.title) or (full_name.first_initial==full_name2.first_initial or full_name.middle_initial==full_name2.middle_initial and full_name.title==full_name2.title):
                            name_holder.append(full_name2)
                            used_indices.append(j)
            used_indices.append(i)
            name_set.append(name_holder)
    
    return name_set                  

In [38]:
def obscure_matches(self, name = 'ex'):
    text_list = list(self.cleaned_text)
    if name == 'pub':
        for (x, y) in [pub.review_loc for pub in self.pub_names]:
            text_list[x:y] = list(len(self.cleaned_text[x:y]) * '@')
    if name == 'person':
        for (x, y) in [pers.review_loc for pers in self.person_names]:
            text_list[x:y] = list(len(self.cleaned_text[x:y]) * '@')
    return ''.join(text_list)

In [39]:
obscure_matches(review_list[35], name = 'pub')

"Book Reviews Tuberculosis or Consumption By Spiers Oberlin Price The subject-matter of this monograph has appeared mainly in the pages of the Lancet-Clinic The author looks upon the disease as one dependent upon the evils of civilization and is in accord with atmospheric environments which may be from within or from without and cause depraved tissue or dyscrasia The Medical Record Visiting ListPhysician's Diary New York @@@@@@@@@@@@@@@@@@@@@ This is an excellent list in fact one of the best of the class of publications that is absolutely necessary for the physician s use being revised and made into modern up-to-date list in every particular The Physicians ' Pocket Account-Book Issued by the Medical Council of Philadelphia This is very practical book little larger and little more complete than the ordinary visiting-list of the past and is intended to answer the purposes of ledger as well as visiting list"

In [40]:
review_list[0].no_pubs_text

'LITERARY NOTES.  - The Chautauqua department of " Wide Awake " is now published separately.  - Mr. Charles Dudley Warner has written for " The North American Review " a paper on prison .  - Mr. Vedder\'s illustrations of the " Rubfilyfit " have found a great admirer in the Queen of Italy.  - @@@@@@@@@@@@@ sill shortly publish a " Dictionary of English, " by Sidney J. Low and F. S. Pulling.  - Professor Thorold Rogers is writing a worL on the " of the Privileges of British Citizenship. "   -   " Mollie Carew " is the title of a pleasant story by A. M. W., just published by @@@@@@@@@@@@@@@@@@, New Yor \' .   -  . - rs. (MISS i ) nas written tne article on Mrs. Browning for the new " Biograpbical Dictionary. "  - Chatto & Windus, the Engllsh publishers, will issue during the coming season the collected essays of Mr. Swinburne.  - Dr. William \' M. Taylor\'s " Life of John Knox " is announced for early by A. C. Armstrong & Sons, New York.  - The " Athenmum " reports that Mark Twain will v

In [57]:
cleaned_text = review_list[0].cleaned_text

In [60]:
text_list = list(cleaned_text)

In [62]:
for (x, y) in [x.review_loc for x in review_list[0].pub_names]:
    print(text_list[x:y])
    print(len(review_list[0].cleaned_text[x:y]) * '@')
    text_list[x:y] = list(len(review_list[0].cleaned_text[x:y]) * '@')

['C', 'a', 's', 's', 'e', 'l', 'l', ' ', '&', ' ', 'C', 'o', '.']
@@@@@@@@@@@@@
['E', '.', ' ', 'P', '.', ' ', 'D', 'u', 't', 't', 'o', 'n', ' ', '&', ' ', 'C', 'o', '.']
@@@@@@@@@@@@@@@@@@
['T', 'h', 'o', 'm', 'a', 's', ' ', 'Y', '.', ' ', 'C', 'r', 'o', 'w', 'e', 'l', 'l', ' ', '&', ' ', 'C', 'o', '.']
@@@@@@@@@@@@@@@@@@@@@@@
['A', 'm', 'e', 'r', 'i', 'c', 'a', 'n', ' ', 'H', 'i', 's', 't', 'o', 'r', 'i', 'c', 'a', 'l', ' ', 'S', 'o', 'c', 'i', 'e', 't', 'y']
@@@@@@@@@@@@@@@@@@@@@@@@@@@
['D', 'u', 'n', 'l', 'a', 'p', ' ', 'S', 'o', 'c', 'i', 'e', 't', 'y']
@@@@@@@@@@@@@@


In [64]:
''.join(text_list)

'LITERARY NOTES.  - The Chautauqua department of " Wide Awake " is now published separately.  - Mr. Charles Dudley Warner has written for " The North American Review " a paper on prison .  - Mr. Vedder\'s illustrations of the " Rubfilyfit " have found a great admirer in the Queen of Italy.  - @@@@@@@@@@@@@ sill shortly publish a " Dictionary of English, " by Sidney J. Low and F. S. Pulling.  - Professor Thorold Rogers is writing a worL on the " of the Privileges of British Citizenship. "   -   " Mollie Carew " is the title of a pleasant story by A. M. W., just published by @@@@@@@@@@@@@@@@@@, New Yor \' .   -  . - rs. (MISS i ) nas written tne article on Mrs. Browning for the new " Biograpbical Dictionary. "  - Chatto & Windus, the Engllsh publishers, will issue during the coming season the collected essays of Mr. Swinburne.  - Dr. William \' M. Taylor\'s " Life of John Knox " is announced for early by A. C. Armstrong & Sons, New York.  - The " Athenmum " reports that Mark Twain will v

In [39]:
def get_names_following_titles(review):
    """
    Returns names following titles - specifically capitalized titles followed by capitalized names.
    Names can be any number of words in length, and can include punctuation.
    
    """
    names = []
    spans = []
    
    txt = review.cleaned_text

    iterx = re.finditer(title_list, txt)
    indices = [(m.start(), m.group()) for m in iterx]
    
    for e, index in enumerate(indices):
        
        if (e==len(indices)-1):
            end_index = -1
        else:
            end_index = indices[e+1][0]
        
        end_span = len(txt[indices[e][0]:end_index])   
        get_match = re.finditer('[A-Z]\w+[^A-Z]|[A-Z].[^A-Z]', txt[indices[e][0]:end_index])
        matches = [(m.span(), m.group()) for m in get_match]
        matches.reverse()
        
        for n, m in enumerate(matches):
            if n<len(matches)-1:
                if (m[0][1] != matches[n-1][0][0]):
                    end_span = m[0][1]
        
        result = txt[indices[e][0]:(indices[e][0] + end_span - 1)]
        
        if len(result) > len(indices[e][1]):
            names.append(txt[indices[e][0]:(indices[e][0] + end_span - 1)])
            spans.append(indices[e][0])
        
    names = [word.replace("'s", "") for word in names]
    names = [PersonName(clean_name(word)) for word in names]
    
    for e, name in enumerate(names):
        name.review_id = review.review_id
        name.review_loc = (spans[e], spans[e]+len(name))
    
    return names

In [46]:
get_names_following_titles(review_list[20])

[Professor Brastow]

## ReviewObject

In [ ]:
# class ReviewObj():
    
#     def __findnames(self):
#         self.pub_names = get_publishers(self)
#         self.person_names = get_names_following_titles(self)
        
#     def __init__(self, aps_id, txt):
#         self.review_id = aps_id
#         self.original_text = txt
#         self.cleaned_text = preprocess_text(txt)
        
#         self.__findnames()

In [ ]:
# review_ex = ReviewObj(136726613, txts[0])

In [ ]:
# for x in review_ex.person_names:
#     print(x)

In [ ]:
# for filename, txt in zip([x.split('.')[0] for x in filenames],txts):
#     rev = ReviewObj(filename, txt)
#     print(filename)
#     if rev.person_names:
#         for x in rev.person_names:
#             print(x)
#         print()

In [ ]:
# for x in review_ex.person_names:
#     print(x)

spacy NLP object steal their code

In [ ]:
# for txt, filename in zip(txts, [x.split('.')[0] for x in filenames]):
#     print(filename)
#     docnames = get_names_following_titles(txt, filename)
#     for name in docnames:
#         print(name, name.review_loc)
#     print()

## Goals
Make each Person name aware of others and able to check for potential matches?